# This notebook contains code examples for maintaining, extending and updating the Wikipathways bot

## load the libraries

In [24]:
from wikidataintegrator import wdi_core, wdi_login
import os
import pandas as pd
import pprint

## Define the functions

In [13]:
def sparql(query, endpoint):
    query=query
    return wdi_core.WDItemEngine.execute_sparql_query(query, endpoint=endpoint)

def sparqlpandas(query, endpoint):
    return wdi_core.WDItemEngine.execute_sparql_query(query, endpoint=endpoint, as_dataframe=True)
    

## Login to the bot
The code snippet below is to login in to Wikidata. First it checks if a local variable containing the wikidata user/bot account and password has been set. In this code example in the jupyter Notebook is use the [%env magic](https://stackoverflow.com/a/44251637/155046). In jenkins the variables are set when configuring the jenkins recipe. 

Running a script like this on Wikidata requires a [bot account](https://www.wikidata.org/wiki/Wikidata:Bots). 


In [9]:
print("Logging in...")
if "WDUSER" in os.environ and "WDPASS" in os.environ:
    WDUSER = os.environ['WDUSER']
    WDPASS = os.environ['WDPASS']
else:
    raise ValueError("WDUSER and WDPASS must be specified in local.py or as environment variables")
login = wdi_login.WDLogin(WDUSER, WDPASS)

Logging in...
https://www.wikidata.org/w/api.php
Successfully logged in as Andrawaag


## Update the description for Pathway http://identifiers.org/wikipathways/WP716

In [19]:
## Find the Wikidata identifier for WP716
query = """
   SELECT * WHERE {?pathway wdt:P2888 <http://identifiers.org/wikipathways/WP716>}
"""
sparqlpandas(query, endpoint="https://query.wikidata.org/sparql")

,pathway
0,http://www.wikidata.org/entity/Q27436284


In [20]:
wdwpid = sparql(query, endpoint="https://query.wikidata.org/sparql")["results"]["bindings"][0]["pathway"]["value"]
print(wdwpid)

http://www.wikidata.org/entity/Q27436284


In [25]:
wdPage = wdi_core.WDItemEngine(wd_item_id=wdwpid.replace("http://www.wikidata.org/entity/", ""))
pprint.pprint(wdPage.get_wd_json_representation())

{'aliases': {},
 'claims': {'P1343': [{'id': 'Q27436284$45ca74de-4520-77f2-a64b-1ae8ce6cf171',
                       'mainsnak': {'datatype': 'wikibase-item',
                                    'datavalue': {'type': 'wikibase-entityid',
                                                  'value': {'entity-type': 'item',
                                                            'id': 'Q27436295',
                                                            'numeric-id': 27436295}},
                                    'property': 'P1343',
                                    'snaktype': 'value'},
                       'qualifiers': {},
                       'qualifiers-order': [],
                       'rank': 'normal',
                       'references': [{'hash': '0e5141c6b678a71885036c39189e6e58c2a524ee',
                                       'snaks': {'P248': [{'datatype': 'wikibase-item',
                                                           'datavalue': {'type': 'wikibase

## Get the description from Wikipathways

In [26]:
query = """
prefix dcterms:	<http://purl.org/dc/terms/>
SELECT ?item ?description WHERE { <"""
query += wdwpid + "> " 
query += """wdt:P2410 ?wpid ;
         wdt:P2888 ?wpiri .
  SERVICE <http://sparql.wikipathways.org/> {
      ?version dc:identifier ?wpiri ;
               dcterms:description ?description .
    }
}
"""
sparqlpandas(query, endpoint="https://query.wikidata.org/sparql")

,description
0,This pathway is about carotenoid and vitamin A...


In [29]:
description = sparql(query, endpoint="https://query.wikidata.org/sparql")["results"]["bindings"][0]["description"]["value"]
print(description)

This pathway is about carotenoid and vitamin A metabolism. The initial version was created by the NuGO focusteam on Carotenoid metabolism. It was used to test a text mining workflow which added some additional entities (see Waagmeester et al. 2009 [https://www.ncbi.nlm.nih.gov/pubmed/19715393]).


In [31]:
wdPage.set_description(description, lang="en")
wdPage.write(login)

Error while writing to Wikidata


WDApiError: {'error': {'code': 'modification-failed', 'info': 'Description must be no more than 250 characters long', 'messages': [{'name': 'wikibase-validator-description-too-long', 'parameters': ['250', 'This pathway is about caroten...'], 'html': {'*': 'Description must be no more than 250 characters long'}}], '*': 'See https://www.wikidata.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}, 'servedby': 'mw1286'}